In [7]:
import cv2
import numpy as np

%config IPCompleter.greedy=True

cap = cv2.VideoCapture(0)
cap.release()
cv2.destroyAllWindows()

In [8]:
def ORB_detector(new_image, image_template):
    image1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    
    orb =  cv2.ORB_create(1000, 1.2)
    
    (kp1, des1) = orb.detectAndCompute(image1, None)
    (kp2, des2) = orb.detectAndCompute(image_template, None)
    
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda val: val.distance)
    return len(matches)

cap = cv2.VideoCapture(0)
image_template = cv2.imread('images/box_in_scene.png', 0)

while True:

    # Get webcam images\n",
    ret, frame = cap.read()
    
    # Get height and width of webcam frame\n",
    height, width = frame.shape[:2]

    # Define ROI Box Dimensions (Note some of these things should be outside the loop)\n",
    top_left_x = int(width / 3)
    top_left_y = int((height / 2) + (height / 4))
    bottom_right_x = int((width / 3) * 2)
    bottom_right_y = int((height / 2) - (height / 4))

    # Draw rectangular window for our region of interest\n",
    cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)
    # Crop window of observation we defined above\n",
    cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

    # Flip frame orientation horizontally\n",
    frame = cv2.flip(frame,1)

    # Get number of ORB matches \n",
    matches = ORB_detector(cropped, image_template)

    # Display status string showing the current no. of matches \n",
    output_string = "Matches = " + str(matches)

    cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

    # Our threshold to indicate object deteciton\n",
    # For new images or lightening conditions you may need to experiment a bit \n",
    # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match\n",
    threshold = 350

    # If matches exceed our threshold then object has been detected\n",
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
        
    cv2.imshow('Object Detector using ORB', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()